In [1]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

from extraction import extract_patches
from reconstruction import perform_voting, generate_indexes

from model import Multimodel

Using TensorFlow backend.


In [12]:
import os
import nibabel as nib

from medical_data import cdr_info, nwbv_info, diff_info

orig_pattern = 'datasets/OASIS/OAS2_RAW_PART1/OAS2_00{0:02}_MR{1}/RAW/mpr-1_sstr_susan_matched.nii.gz'
prob_pattern = 'datasets/OASIS/OAS2_RAW_PART1/OAS2_00{0:02}_MR{1}/RAW/mpr-1_sstr_susan_pve_{2}.nii.gz'

step = (64, 64)
threshold = np.int32(0.50 * np.prod(curr_patch_shape[:]))
X_train = np.empty((0, 3, ) + curr_patch_shape)
Y_train = np.empty((0, 1, ) + curr_patch_shape)
for i in range(1, 100) :
    print '{} :'.format(i),
    for j in range(1, 6) :
        filename = orig_pattern.format(i, j)
        
        if not os.path.exists(filename) :
            continue

        volume_init = nib.load(filename).get_data()
        volume_mask = volume_init != 0
        mask_patches = extract_patches(volume_mask, (1, ) + curr_patch_shape, (1, ) + step)
        
        useful_patches = np.sum(mask_patches, axis=(1, 2, 3)) > threshold
        
        N = np.sum(useful_patches)

        volume_init = (volume_init - volume_init.mean()) / volume_init.std()

        ref_patches = extract_patches(volume_init, (1, ) + curr_patch_shape, (1, ) + step)
        ref_patches = ref_patches[useful_patches].reshape((-1, 1, ) + curr_patch_shape)
        
        Y_train = np.vstack((ref_patches, Y_train))
        
        del ref_patches
        
        new_patches = np.empty((N, 3, ) + curr_patch_shape)
        for k in range(3) :
            filename = prob_pattern.format(i, j, k)
            prob_mask_init = nib.load(filename).get_data()
            prob_patches = extract_patches(prob_mask_init, (1, ) + curr_patch_shape, (1, ) + step)
            prob_patches = prob_patches[useful_patches].reshape((-1, ) + curr_patch_shape)
            
            new_patches[:, k] = prob_patches
            
            del prob_patches
            
        X_train = np.vstack((new_patches, X_train))
        
        del new_patches
        
        print j,
    print

1 : 1 2
2 : 1 2 3
3 :
4 : 1 2
5 : 1 2 3
6 :
7 : 1 3 4
8 : 1 2
9 : 1 2
10 : 1 2
11 :
12 : 1 2 3
13 : 1 2 3
14 : 1 2
15 :
16 : 1 2
17 : 1 3 4 5
18 : 1 3 4
19 :
20 : 1 2 3
21 : 1 2
22 : 1 2
23 : 1 2
24 :
25 :
26 : 1 2
27 : 1 2 3 4
28 : 1 2
29 : 1 2
30 : 1 2
31 : 1 2 3
32 : 1 2
33 :
34 : 1 2 3 4
35 : 1 2
36 : 1 3 4 5
37 : 1 2 3 4
38 :
39 : 1 2
40 : 1 2 3
41 : 1 2 3
42 : 1 2
43 : 1 2
44 : 1 2 3
45 : 1 2
46 : 1 2
47 : 1 2
48 : 1 2 3 4 5
49 : 1 2 3
50 : 1 2
51 : 1 2 3
52 : 1 2
53 : 1 2
54 : 1 2
55 : 1 2
56 : 1 2
57 : 1 2 3
58 : 1 2 3
59 :
60 : 1 2
61 : 1 2 3
62 : 1 2 3
63 : 1 2
64 : 1 2 3
65 :
66 : 1 2
67 : 1 2 3 4
68 : 1 2
69 : 1 2
70 : 1 2 3 4 5
71 : 1 2
72 :
73 : 1 2 3 4 5
74 :
75 : 1 2
76 : 1 2 3
77 : 1 2
78 : 1 2 3
79 : 1 2 3
80 : 1 2 3
81 : 1 2
82 :
83 :
84 :
85 : 1 2
86 : 1 2
87 : 1 2
88 : 1 2
89 : 1 3
90 : 1 2 3
91 : 1 2
92 : 1 2
93 :
94 : 1 2
95 : 1 2 3
96 : 1 2
97 : 1 2
98 : 1 2
99 : 1 2


In [44]:
curr_patch_shape = (128, 128)
input_modalities = ['seg', 'T1']
output_modalities = ['Gen']
output_weights = {'Gen' : 1.0, 'concat' : 1.0}
latent_dim = 16
channels = [3, 1]
use_dropout = [False, True]
patch_shape = curr_patch_shape
scale = 1
a_model = Multimodel(
    input_modalities, output_modalities, output_weights, latent_dim, channels, patch_shape, use_dropout, scale)
a_model.build()

Latent dimensions: 16
Fuse latent representations using max
making output: Tensor("enc_seg_act9_5/add:0", shape=(?, 16, 128, 128), dtype=float32) Tensor("dec_Gen_18/dec_Gen_act5/add:0", shape=(?, 1, 128, 128), dtype=float32) em_0_dec_Gen
making output: Tensor("dropout_7/cond/Merge:0", shape=(?, 16, 128, 128), dtype=float32) Tensor("dec_Gen_19/dec_Gen_act5/add:0", shape=(?, 1, 128, 128), dtype=float32) em_1_dec_Gen
making output: Tensor("combined_em_6/Maximum:0", shape=(?, 16, 128, 128), dtype=float32) Tensor("dec_Gen_20/dec_Gen_act5/add:0", shape=(?, 1, 128, 128), dtype=float32) em_2_dec_Gen
making output: em_concat Tensor("em_concat_6/concat:0", shape=(?, 2, ?), dtype=float32) em_concat_6/concat:0
all outputs:  [u'em_0_dec_Gen_6/add:0', u'em_1_dec_Gen_6/add:0', u'em_2_dec_Gen_6/add:0', u'em_concat_6/concat:0', u'em_fused_6/ExpandDims:0']
output dict:  {'em_concat': <function embedding_distance at 0x7f9caf0d55f0>, 'em_fused': <function embedding_distance at 0x7f9caf0d55f0>, 'em_1_dec_G

In [45]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

patience = 2

stopper = EarlyStopping(patience=patience)
checkpointer = ModelCheckpoint('model.h5', save_best_only=True, save_weights_only=True)

N = len(X_train)
a_model.model.fit(
    [X_train, Y_train],
    [Y_train, Y_train, Y_train, np.empty((N, 2, 0)), np.empty((N, 1, 0))],
    validation_split=0.3, epochs=20,
    callbacks=[checkpointer, stopper])

Train on 8618 samples, validate on 3694 samples
Epoch 1/20
8618/8618 [==============================] - 137s 16ms/step - loss: 0.4782 - em_0_dec_Gen_loss: 0.2034 - em_1_dec_Gen_loss: 0.1365 - em_2_dec_Gen_loss: 0.1194 - em_concat_loss: 0.0188 - em_fused_loss: 0.0000e+00 - val_loss: 0.5335 - val_em_0_dec_Gen_loss: 0.1586 - val_em_1_dec_Gen_loss: 0.1884 - val_em_2_dec_Gen_loss: 0.1827 - val_em_concat_loss: 0.0038 - val_em_fused_loss: 0.0000e+00
Epoch 2/20
8618/8618 [==============================] - 119s 14ms/step - loss: 0.2425 - em_0_dec_Gen_loss: 0.1290 - em_1_dec_Gen_loss: 0.0581 - em_2_dec_Gen_loss: 0.0489 - em_concat_loss: 0.0066 - em_fused_loss: 0.0000e+00 - val_loss: 0.5144 - val_em_0_dec_Gen_loss: 0.1408 - val_em_1_dec_Gen_loss: 0.2004 - val_em_2_dec_Gen_loss: 0.1708 - val_em_concat_loss: 0.0024 - val_em_fused_loss: 0.0000e+00
Epoch 3/20
8618/8618 [==============================] - 119s 14ms/step - loss: 0.2075 - em_0_dec_Gen_loss: 0.1138 - em_1_dec_Gen_loss: 0.0475 - em_2_dec_G

In [38]:
a_model.model.load_weights('model.h5')

In [39]:
from keras.models import Model
import os
import nibabel as nib
import SimpleITK as sitk

orig_pattern = 'datasets/OASIS/OAS2_RAW_PART1/OAS2_00{0:02}_MR{1}/RAW/mpr-1_sstr.nii.gz'
prob_pattern = 'datasets/OASIS/OAS2_RAW_PART1/OAS2_00{0:02}_MR{1}/RAW/mpr-1_sstr_susan_pve_{2}.nii.gz'

step = (32, 32)
for i in range(98, 100) :
    filename = orig_pattern.format(i, 1)
    if not os.path.exists(filename) :
        continue
    
    print i

    volume_data = nib.load(filename)
    volume_actual = volume_data.get_data()
    volume_init = volume_actual != 0
    ref_patches = extract_patches(volume_init, (1, ) + curr_patch_shape, (1, ) + step)
    ref_patches = ref_patches.reshape((-1, 1, ) + curr_patch_shape)
    
    N = len(ref_patches)
    new_patches = np.empty((N, 3, ) + curr_patch_shape)
    for k in range(3) :
        filename = prob_pattern.format(i, 1, k)
        prob_mask_init = nib.load(filename).get_data()
        prob_patches = extract_patches(prob_mask_init, (1, ) + curr_patch_shape, (1, ) + step)
        prob_patches = prob_patches.reshape((-1, ) + curr_patch_shape)

        new_patches[:, k] = prob_patches

        del prob_patches
    
    pred = a_model.model.predict([new_patches, np.zeros_like(new_patches[:, 0:1, :, :])], verbose=1)[0]

    volume = perform_voting(pred.reshape((-1, 1, ) + curr_patch_shape), (1, ) + curr_patch_shape, (256, 256, 128), (1, ) + step)
    
    volume[volume_actual != 0] = volume[volume_actual != 0] * volume_actual[volume_actual != 0].std() #+ volume_actual[volume_actual != 0].mean()
    
    volume = np.multiply(volume_data.get_data() != 0, volume)

    nib.save(nib.Nifti1Image(volume, volume_data.affine), 'result_{}.nii.gz'.format(i))
    
    del new_patches, ref_patches
    
    res = sitk.ReadImage('result_{}.nii.gz'.format(i))

    caster = sitk.CastImageFilter()
    caster.SetOutputPixelType(res.GetPixelID())

    orig = caster.Execute(sitk.ReadImage(orig_pattern.format(i, 1)))

    matcher = sitk.HistogramMatchingImageFilter()
    matcher.SetNumberOfHistogramLevels(1024)
    matcher.SetNumberOfMatchPoints(15)
    matched = matcher.Execute(res, orig)

    sitk.WriteImage(matched, 'result_cor_{}.nii.gz'.format(i))

98
1280/1280 [==============================] - 9s 7ms/step
99
1280/1280 [==============================] - 8s 6ms/step
